<a href="https://colab.research.google.com/github/Thomasula/practise/blob/main/code/01RAD_Ex07_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01RAD Exercise 7 - team work

Authors: name1, name 2, name3


## Description of the Assignment

The dataset `Boston` contains a total of 506 records from towns in the suburbs of Boston, MA, USA. The data originates from the study by Harrison, D., and Rubinfeld, D.L. (1978), *Hedonic prices and the demand for clean air*, J. Environ. Economics and Management, 5, 81–102.

The dataset includes 14 variables. The goal is to explore the influence of 13 of them on the median value of owner-occupied homes (`medv`). Below is a description of the variables:

| Feature   | Description                                                                 |
|-----------|-----------------------------------------------------------------------------|
| `crim`    | Per capita crime rate by town                                              |
| `zn`      | Proportion of residential land zoned for lots over 25,000 sq.ft            |
| `indus`   | Proportion of non-retail business acres per town                           |
| `chas`    | Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)      |
| `nox`     | Nitrogen oxides concentration (parts per 10 million)                       |
| `rm`      | Average number of rooms per dwelling                                       |
| `age`     | Proportion of owner-occupied units built prior to 1940                     |
| `dis`     | Weighted mean of distances to five Boston employment centres               |
| `rad`     | Index of accessibility to radial highways                                  |
| `tax`     | Full-value property-tax rate per $10,000$                                   |
| `ptratio` | Pupil-teacher ratio by  town    |                                            |
| `black_tra`   | $1000\left(\text{black_pop} - 0.63\right)^2$ where `black_pop` is the proportion of blacks by town       |
| `lstat`   | Lower status of the population (percent)                                   |
| `medv`    | Median value of owner-occupied homes in $1000s                             |

---

## Conditions and Scoring

- Collaboration in the team is allowed and recommended.
- This homework includes 14 questions.
- Submit the homework in the corresponding `.ipynb` file, via MS Teams by the next week.
---


In [ ]:
# Import libraries
import pandas as pd
import numpy as np


In [ ]:
import pandas as pd
import numpy as np

# URL for the Boston housing dataset
data_url = "http://lib.stat.cmu.edu/datasets/boston"

# Reading the dataset
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)

# Processing the dataset into features and target
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

# Column names
columns = [
    "crim", "zn", "indus", "chas", "nox", "rm", "age",
    "dis", "rad", "tax", "ptratio", "black_tra", "lstat"
]
boston_df = pd.DataFrame(data, columns=columns)
boston_df["medv"] = target


boston_df


## Exploratory and Graphical Analysis

### Question 01

- Check for missing values and verify the dimensions of the dataset.
- Summarize the descriptive statistics of all variables.
- Plot a histogram and density estimate for the response variable `medv`.
- Examine the frequency table of `medv` values and discuss whether rounding, truncation, or other issues are present.
- Remove measurements deemed unreliable and discuss what this implies for the response model.

```python





In [ ]:
boston_df.info()
boston_df.describe()
print(boston_df.isnull().sum())  # Check for missing values
print(boston_df.shape)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10, 6))
sns.histplot(boston_df['medv'], kde=True, bins=30, color='blue', edgecolor='black')
plt.title("Histogram and Density Estimate for MEDV")
plt.xlabel("MEDV")
plt.ylabel("Frequency")
plt.show()

In [ ]:
print(boston_df['medv'].value_counts().sort_index())
unreliable_measurements = boston_df[boston_df['medv'] == 50].index
df_cleaned = boston_df.drop(unreliable_measurements)

print(f"Removed {len(unreliable_measurements)} unreliable measurements.")

plt.figure(figsize=(10, 6))
sns.histplot(df_cleaned['medv'], kde=True, bins=30, color='orange', edgecolor='black')
plt.title("Histogram and Density Estimate for MEDV")
plt.xlabel("MEDV")
plt.ylabel("Frequency")
plt.show()

Measurements with 'meadv' >= 50 removed. Model will be reliable for data with 'meadv' under 50 or only slightly above 50.

## Simple Regression Model: Median Price and Crime

### Question 2

- Build a simple linear regression model to examine if the crime rate (`crim`) affects the median value of homes (`medv`).
- If there is an effect, determine how much the housing price decreases as the crime rate increases.

---

### Question 3

- Experiment with power and logarithmic transformations of the response variable (`medv`).
- To find the optimal power transformation, plot the log-likelihood profile for the Box-Cox transformation and compare it with a logarithmic transformation.

---

### Question 4

- Based on the simple linear model and on the model with logarithmic transformations of the response variable, estimate the increase or decrease in housing prices for a one-unit change in the crime rate (`crim`).
- Provide the correct interpretation from both models.

---

### Question 5

- Keep the logarithmic transformation of the response (`medv`) and try transforming the independent variable (`crim`).
- Use techniques such as piecewise constant transformations, or polynomial transformations (quadratic and cubic).
- Use information from plots such as Component-Residual Plots (Partial Residual Plots) and Partial Regression Plots to guide your transformations.
- Discuss whether these models can be compared using an F-test. If applicable, perform the test and interpret the results.

---

### Question 6

- Select one of the previous models, justify your choice, and validate it using the appropriate hypothesis tests for residuals (normality, homoscedasticity, etc.).
- Use diagnostic plots such as Q-Q plots, residuals vs. fitted values, and others to evaluate the model's assumptions.

---


In [ ]:
formula = 'medv ~ (crim)'

# Fit an OLS model with interactions
model = smf.ols(formula, data=df_cleaned).fit()

# Display model summary
print(model.summary())

Model with intercept can be interpreted as how the home price varies if there is higher crime rate from the mean home price. The R2 statistics is quite low, although for crude data and only one parameter it is not bad.
The model shows how the median house price varies from the price when crime rate is 0, then for every unit the crime rate increases, house value drops by $400.


In [ ]:
df_cleaned['log_medv'] = np.log(df_cleaned['medv'])
sns.histplot(df_cleaned['log_medv'], kde=True, bins=30, color='blue', edgecolor='black')
plt.title("Histogram of Log-Transformed MEDV")
plt.xlabel("log(MEDV)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
from scipy.stats import boxcox
from scipy.stats import boxcox_llf
from scipy.stats import chi2

df_cleaned['medv_positive'] = df_cleaned['medv'] + 1e-5
boxcox_transformed, lambda_opt = boxcox(df_cleaned['medv_positive'])

df_cleaned['boxcox_medv'] = boxcox_transformed

print(f"Optimal lambda for Box-Cox Transformation: {lambda_opt}")

lambda_values = np.linspace(-2, 2, 100)

log_likelihoods = [boxcox_llf(lmb, df_cleaned['medv_positive']) for lmb in lambda_values]

max_log_likelihood = max(log_likelihoods)

threshold = max_log_likelihood - chi2.ppf(0.95, df=1) / 2

lambda_conf_interval = lambda_values[
    (log_likelihoods >= threshold)
]

lambda_low = min(lambda_conf_interval)
lambda_high = max(lambda_conf_interval)

plt.figure(figsize=(10, 6))
plt.plot(lambda_values, log_likelihoods, label='Log-Likelihood')
plt.axvline(lambda_opt, color='red', linestyle='--', label=f"Optimal lambda = {lambda_opt:.2f}")
plt.axvline(lambda_low, color="green", linestyle="--", label=f"95% CI Lower = {lambda_low:.2f}")
plt.axvline(lambda_high, color="blue", linestyle="--", label=f"95% CI Upper = {lambda_high:.2f}")
plt.axhline(threshold, color="gray", linestyle="--", label="95% CI Threshold")
plt.title("Log-Likelihood Profile for Box-Cox Transformation")
plt.xlabel("Lambda")
plt.ylabel("Log-Likelihood")
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.histplot(df_cleaned['log_medv'], kde=True, color='blue', bins=30)
plt.title("Logarithmic Transformation (lambda = 0)")
plt.xlabel("log(MEDV)")

plt.subplot(1, 2, 2)
sns.histplot(df_cleaned['boxcox_medv'], kde=True, color='green', bins=30)
plt.title(f"Box-Cox Transformation (Optimal lambda = {lambda_opt:.2f})")
plt.xlabel("Box-Cox Transformed MEDV")

plt.tight_layout()
plt.show()

Lambda 0.5 was chosen as it is close to 0.4 and easier to apply and interpret

In [ ]:
import statsmodels.api as sm

X = sm.add_constant(df_cleaned['crim'])

model_log = sm.OLS(df_cleaned['log_medv'], X).fit()

model_boxcox = sm.OLS(df_cleaned['boxcox_medv'], X).fit()

print("Logarithmic Transformation Model Summary:")
print(model_log.summary())

print(f"\nBox-Cox Transformation with lambda = {lambda_opt:.2f} Model Summary:")
print(model_boxcox.summary())


In [ ]:
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.histplot(model_log.resid, kde=True, color='blue')
plt.title("Residuals of Log-Transformed Model")

plt.subplot(1, 2, 2)
sns.histplot(model_boxcox.resid, kde=True, color='green')
plt.title("Residuals of Box-Cox-Transformed Model")

plt.tight_layout()
plt.show()

The price change under the model with just intercept and cirme rate, without data transformation was already explained. If we perform the logarithmic transformation, the crime rate coefficient has to be interpreted as a percentage change, that is calculated as: 100*(e^{beta_crime} - 1)
because the values need to be transromed back using the exponential and then interpreted as proportional change

In [ ]:
df_cleaned['crim_binned'] = pd.cut(df_cleaned['crim'], bins=[-np.inf, 1, 5, 10, np.inf], labels=['low', 'medium', 'high', 'very_high'])

crim_dummies = pd.get_dummies(df_cleaned['crim_binned'], drop_first=True)
X_piecewise = sm.add_constant(crim_dummies)

model_piecewise = sm.OLS(df_cleaned['log_medv'], X_piecewise.astype(float)).fit()
print(model_piecewise.summary())

In [ ]:
df_cleaned['crim_squared'] = df_cleaned['crim'] ** 2
df_cleaned['crim_cubed'] = df_cleaned['crim'] ** 3

X_poly2 = sm.add_constant(df_cleaned[['crim', 'crim_squared']])
model_poly2 = sm.OLS(df_cleaned['log_medv'], X_poly2.astype(float)).fit()

X_poly3 = sm.add_constant(df_cleaned[['crim', 'crim_squared', 'crim_cubed']])
model_poly3 = sm.OLS(df_cleaned['log_medv'], X_poly3.astype(float)).fit()

print("Quadratic Model Summary:")
print(model_poly2.summary())

print("\nCubic Model Summary:")
print(model_poly3.summary())

Even higher order terms seem to be statistically significant and R2 and even adj-R2 improved when moving to cubed variables. Other statistics remained rather same.

In [ ]:
from statsmodels.graphics.regressionplots import plot_ccpr
from statsmodels.graphics.regressionplots import plot_partregress

fig, ax = plt.subplots(figsize=(8, 6))
plot_ccpr(model_poly2, 'crim', ax=ax)
plt.title("Component-Residual Plot for crim (Quadratic Model)")
plt.show()

fig, ax = plt.subplots(figsize=(8, 6))
plot_partregress('log_medv', 'crim', ['crim_squared'], data=df_cleaned, ax=ax)
plt.title("Partial Regression Plot for crim (Quadratic Model)")
plt.show()

Null hypothesis: linear model is sufficient vs. the higher orders have large significance. Since the squared and cubed crime had good statistical significance it is suitable to compare them using the F statistics.

In [ ]:
# Compare linear model to quadratic model
f_test_result = model_poly2.compare_f_test(model)
print(f"F-test result (linear vs. quadratic): {f_test_result}")

# Compare linear to cubic model
f_test_result2 = model_poly3.compare_f_test(model)
print(f"F-test result (quadratic vs. cubic): {f_test_result2}")

linear vs. quadratic results: F=288844.00, p=0.0, df=1.0, which indicatec that the cubic model fits significantly better than the linar one, however the improvement is not large going from quadratic to cubic model.

In [ ]:
# Compare quadratic to cubic
f_test_result3 = model_poly3.compare_f_test(model_poly2)
print(f"F-test result (quadratic vs. cubic): {f_test_result3}")

In [ ]:
# Residuals vs. Fitted Values
plt.figure(figsize=(8, 6))
residuals_poly2 = model_poly2.resid
sns.residplot(x=model_poly2.fittedvalues, y=residuals_poly2, lowess=True, line_kws={'color': 'red'})
plt.title('Residuals for degree 2 model vs Fitted Values')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.show()

In [ ]:
# Residuals vs. Fitted Values
plt.figure(figsize=(8, 6))
residuals_poly3 = model_poly3.resid
sns.residplot(x=model_poly3.fittedvalues, y=residuals_poly3, lowess=True, line_kws={'color': 'red'})
plt.title('Residuals for degree 2 model vs Fitted Values')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.show()

In [ ]:
import scipy.stats as stats

# Q-Q plot
plt.figure(figsize=(8, 6))
stats.probplot(residuals_poly2, dist="norm", plot=plt)
plt.title('Q-Q Plot')
plt.show()

In [ ]:
import scipy.stats as stats

# Q-Q plot
plt.figure(figsize=(8, 6))
stats.probplot(residuals_poly3, dist="norm", plot=plt)
plt.title('Q-Q Plot')
plt.show()

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan

X_quad_const = sm.add_constant(df_cleaned[['crim', 'crim_squared']])

bp_test_stat, bp_test_p_value, _, _ = het_breuschpagan(residuals_poly2, X_quad_const)
print(f"Breusch-Pagan test statistic: {bp_test_stat}, p-value: {bp_test_p_value}")

X_quad_const = sm.add_constant(df_cleaned[['crim', 'crim_squared']])

bp_test_stat, bp_test_p_value, _, _ = het_breuschpagan(residuals_poly3, X_quad_const)
print(f"Breusch-Pagan test statistic: {bp_test_stat}, p-value: {bp_test_p_value}")

By simply observing the plotted residuals one could see that there is no signifficant heteroskedacity. I added the Breusch-Pagan test here for completeness, where we failed to reject the null hypothesis that there is heteroskedacity. Durbin-Watson statistics for both models was around 0.8, which suggests some autocorrelation (which appears with the Durbin-Watson statistics being near 0).

Based on D-W statistics, QQ plots and residual plots being largely the same with the quadratic and the cubic model and seeing no significant improvement taking the cubic model... R2 didn't improve much and the F-test suggested that although there is statistical signifficance in taking the cubic model, the improvement is not worth it. I would pick the quadratic model, since the interpretation is more straight forward and it is simpler and also offers good precision.


## Multivariate Regression Model

### Question 7

- Build a multivariate linear regression model with a logarithmic transformation of the response (`medv`).
- Explore relationships between housing prices and other independent variables in an additive model (no interactions).
- Use criteria such as AIC, BIC, $ R^2 $, and F-statistics to select the best model.
- Investigate whether the relationship between `crim` and `medv` can be explained by other variables, such as proximity to highways or pollution levels.

---

### Question 8

- Incorporate `crim` (crime rate) into the final model and compare how its influence on the median housing price differs from the simple regression model with a logarithmic transformation of the response (from Question 4).
- Estimate the reduction in median housing price for a one-unit increase in the crime rate per 1,000 residents.

---

### Question 9

- Present your final predictive model for `medv` and discuss the key parameters such as $ R^2 $, $ \sigma $, and F-statistics.
- Compare the final model with the simple linear model from Question 6. Discuss how these parameters have changed and whether this change was expected.
- Validate the model both graphically and using hypothesis tests.

---

### Question 10

- Based on your final model, answer whether reducing the crime rate in an area would lead to an increase in housing prices in that area.
- Provide an explanation based on your findings.


## Investigating the Transformation of the `black_tra` Variable

<!--
# Add a new variable `black_pop` representing the proportion of Black population
boston_df["black_pop"] = 0.63 - np.sqrt(boston_df["black_tra"] / 1000)
 -->

In [ ]:
# Motivation of this section
from sklearn.datasets import load_boston

# Load dataset
boston_data = load_boston()


### Question 11: Compare Coefficients in Simple Models

Investigate, if the transformation of `black_pop` into `black_tra` was  misleading and suggestive. Add new variable `black_pop` into the data frame by inverse of orginal transformation.

- Build two separate simple linear regression models:
  1. Predicting `medv` using `black_tra`.
  2. Predicting `medv` using `black_pop`.
- Compare the coefficients from both models and interpret the differences.
- Discuss whether the transformation of `black_tra` appears to exaggerate or diminish its relationship with `medv`.

---

### Question 12: Stepwise Regression with `black_tra`

- Perform stepwise regression starting with all independent variables, including `black_tra`, as predictors of `medv`.
- Evaluate whether `black_tra` remains significant in the final model after stepwise variable selection.
- Discuss whether its significance changes when considered alongside other predictors.

---

### Question 13: Stepwise Regression with `black_pop`

- Repeat the stepwise regression from Question 12, but this time replace `black_tra` with `black_pop`.
- Evaluate whether `black_pop` remains significant in the final model.
- Compare its significance to that of `black_tra` from Question 12.

---

### Question 14: Impact on Predictions

- For both the models from Questions 12 and 13 (stepwise regression with `black_tra` and `black_pop`), compare their predictions for `medv`.
- Specifically:
  1. Calculate predictions for a range of values of `black_tra` and `black_pop`.
  2. Plot the predictions and interpret whether the two variables result in substantially different predicted values.
- Discuss whether the transformed variable (`black_tra`) or its proportion counterpart (`black_pop`) leads to any noticeable bias or distortion in predictions.
